Para filtrar por semestre:
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    *
FROM
    juros_mensal;
WHERE
    data LIKE '%-06%' OR data LIKE '%-12%';
"""

juros = pd.read_sql_query(query, conn)

conn.close()

In [1]:
# juros_consignado_privado

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    *
FROM
    juros_consignado_privado
;
"""

juros = pd.read_sql_query(query, conn)

conn.close()

In [2]:
# cdi_mensal

conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    *
FROM
    cdi_mensal
;
"""

cdi = pd.read_sql_query(query, conn)

conn.close()

In [3]:
# Criando tabela model

cdi = cdi[['data', 'cdi', 'selic']]
model = pd.merge(juros, cdi, on='data')

model['TaxaJurosAoAno'] = model['TaxaJurosAoAno'].str.replace(',', '.', regex=False)
model['TaxaJurosAoAno'] = model['TaxaJurosAoAno'].astype(float)
model['cdi'] = model['cdi'].str.replace(',', '.', regex=False)
model['cdi'] = model['cdi'].astype(float)
model['selic'] = model['selic'].str.replace(',', '.', regex=False)
model['selic'] = model['selic'].astype(float)
model['spread'] = model['TaxaJurosAoAno'] - model['cdi']


In [4]:
# Operações de Crédito - Conta 16000001 (filtragem maiores saldos)

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancetes
    WHERE
        "CONTA" = '16000001'
)
SELECT
    *
FROM
    RankedData
WHERE
    rn <= 10;
"""

operações_de_credito = pd.read_sql_query(query, conn)

conn.close()

operações_de_credito = operações_de_credito[['data', 'cnpj', 'SALDO']]
operações_de_credito['cnpj'] = operações_de_credito['cnpj'].astype(int)
operações_de_credito = operações_de_credito.rename(columns={'SALDO': 'operacoes_de_credito'})

model = pd.merge(
    model,
    operações_de_credito,
    on=['data', 'cnpj'],
    how='left'
)
model.dropna(subset=['operacoes_de_credito'], inplace=True)


In [5]:
# Ativo Circulante e Realizável à Longo Prazo - Conta 10000007

import numpy as np
conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '10000007'
"""

ativo_realizavel = pd.read_sql_query(query, conn)

conn.close()

ativo_realizavel = ativo_realizavel[['data', 'cnpj', 'SALDO']]
ativo_realizavel['cnpj'] = (ativo_realizavel['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
ativo_realizavel.dropna(subset=['cnpj'], inplace=True)
ativo_realizavel['cnpj'] = ativo_realizavel['cnpj'].astype(int)
ativo_realizavel = ativo_realizavel.rename(columns={'SALDO': 'ativo_realizavel'})

model = pd.merge(
    model,
    ativo_realizavel,
    on=['data', 'cnpj'],
    how='left'
)

In [6]:
# Depósitos à Vista (Depósitos Não Remunerados) - Conta 41100000

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '41100000'
"""

depositos_a_vista = pd.read_sql_query(query, conn)

conn.close()

depositos_a_vista = depositos_a_vista[['data', 'cnpj', 'SALDO']]
depositos_a_vista['cnpj'] = depositos_a_vista['cnpj'].astype(int)
depositos_a_vista = depositos_a_vista.rename(columns={'SALDO': 'depositos_a_vista_total'})

model = pd.merge(
    model,
    depositos_a_vista,
    on=['data', 'cnpj'],
    how='left'
)
model['depositos_a_vista'] = model['depositos_a_vista_total'] / model['ativo_realizavel']

In [7]:
# Depósitos - Conta 41000007

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '41000007'
"""

depositos = pd.read_sql_query(query, conn)

conn.close()

depositos = depositos[['data', 'cnpj', 'SALDO']]
depositos['cnpj'] = depositos['cnpj'].astype(int)
depositos = depositos.rename(columns={'SALDO': 'depositos'})

model = pd.merge(
    model,
    depositos,
    on=['data', 'cnpj'],
    how='left'
)


In [8]:
# Ativo Permanente - Conta 20000004

import numpy as np
conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '20000004'
"""

ativo_permanente = pd.read_sql_query(query, conn)

conn.close()

ativo_permanente = ativo_permanente[['data', 'cnpj', 'SALDO']]
ativo_permanente['cnpj'] = (ativo_permanente['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
ativo_permanente.dropna(subset=['cnpj'], inplace=True)
ativo_permanente['cnpj'] = ativo_permanente['cnpj'].astype(int)
ativo_permanente = ativo_permanente.rename(columns={'SALDO': 'ativo_permanente'})

model = pd.merge(
    model,
    ativo_permanente,
    on=['data', 'cnpj'],
    how='left'
)
model['ativo_total'] = model['ativo_permanente'] + model['ativo_realizavel']


In [9]:
# Patrimônio Líquido - Conta 60000002

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '60000002'
"""

patrimonio_liquido = pd.read_sql_query(query, conn)

conn.close()

patrimonio_liquido = patrimonio_liquido[['data', 'cnpj', 'SALDO']]
patrimonio_liquido['cnpj'] = (patrimonio_liquido['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
patrimonio_liquido.dropna(subset=['cnpj'], inplace=True)
patrimonio_liquido['cnpj'] = patrimonio_liquido['cnpj'].astype(int)
patrimonio_liquido = patrimonio_liquido.rename(columns={'SALDO': 'patrimonio_liquido'})

model = pd.merge(
    model,
    patrimonio_liquido,
    on=['data', 'cnpj'],
    how='left'
)


In [10]:
# Passivo Total -  Conta 99999995

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '99999995'
"""

passivo_total = pd.read_sql_query(query, conn)

conn.close()

passivo_total = passivo_total[['data', 'cnpj', 'SALDO']]
passivo_total['cnpj'] = (passivo_total['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
passivo_total.dropna(subset=['cnpj'], inplace=True)
passivo_total['cnpj'] = passivo_total['cnpj'].astype(int)
passivo_total = passivo_total.rename(columns={'SALDO': 'passivo_total'})

model = pd.merge(
    model,
    passivo_total,
    on=['data', 'cnpj'],
    how='left'
)


In [11]:
# Alavancagem (lever)

model['lever'] = model['passivo_total'] / model['patrimonio_liquido']

In [12]:
# Receita Operacional - Conta 71000008

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '71000008'
"""

receita_operacional = pd.read_sql_query(query, conn)

conn.close()

receita_operacional = receita_operacional[['data', 'cnpj', 'SALDO']]
receita_operacional['cnpj'] = (receita_operacional['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
receita_operacional.dropna(subset=['cnpj'], inplace=True)
receita_operacional['cnpj'] = receita_operacional['cnpj'].astype(int)
receita_operacional = receita_operacional.rename(columns={'SALDO': 'receita_operacional'})

model = pd.merge(
    model,
    receita_operacional,
    on=['data', 'cnpj'],
    how='left'
)

In [13]:
# Despesa Operacional - Conta 81000005

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '81000005'
"""

despesa_operacional = pd.read_sql_query(query, conn)

conn.close()

despesa_operacional = despesa_operacional[['data', 'cnpj', 'SALDO']]
despesa_operacional['cnpj'] = (despesa_operacional['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
despesa_operacional.dropna(subset=['cnpj'], inplace=True)
despesa_operacional['cnpj'] = despesa_operacional['cnpj'].astype(int)
despesa_operacional = despesa_operacional.rename(columns={'SALDO': 'despesa_operacional'})

model = pd.merge(
    model,
    despesa_operacional,
    on=['data', 'cnpj'],
    how='left'
)


In [14]:
# Fundos Remunerados
'''
Depósitos de Poupança (4.1.2.00.00-3)
Depósitos Interfinanceiros (4.1.3.00.00-6)
Depósitos a Prazo (4.1.5.00.00-2)
Depósitos sob Aviso (4.1.4.00.00-9)
Obrigações por Operações Compromissadas (4.2.0.00.00-6)
Obrigações por Títulos e Valores Mobiliários (4.3.0.00.00-5)
Obrigações por Empréstimos e Repasses (4.6.0.00.00-2)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('41200003', '41300006', '41500002', '41400009', '42000006', '43000005', '46000002')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

fundos_remunerados = pd.read_sql_query(query, conn)
conn.close()

fundos_remunerados['cnpj'] = fundos_remunerados['cnpj'].astype(int)
fundos_remunerados = fundos_remunerados.rename(columns={'saldo_total_agregado': 'fundos_remunerados'})

model = pd.merge(
    model,
    fundos_remunerados,
    on=['data', 'cnpj'],
    how='left'
)


In [15]:
# Ativos rentáveis
'''
Aplicações Interfinanceiras de Liquidez (1.2.0.00.00-5)
Títulos e Valores Mobiliários e Instrumentos Financeiros Derivativos (1.3.0.00.00-4)
Operações de Crédito (1.6.0.00.00-1)
Operações de Arrendamento Mercantil (1.7.0.00.00-0)
Outros Créditos (1.8.0.00.00-9)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('12000005', '13000004', '16000001', '17000000', '18000009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

ativos_rentaveis = pd.read_sql_query(query, conn)
conn.close()

ativos_rentaveis['cnpj'] = (ativos_rentaveis['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
ativos_rentaveis.dropna(subset=['cnpj'], inplace=True)
ativos_rentaveis['cnpj'] = ativos_rentaveis['cnpj'].astype(int)
ativos_rentaveis = ativos_rentaveis.rename(columns={'saldo_total_agregado': 'ativos_rentaveis'})

model = pd.merge(
    model,
    ativos_rentaveis,
    on=['data', 'cnpj'],
    how='left'
)


In [16]:
# ibf (Interest Bearing Funds / Total Earning Assets)

model['ibf'] = model['fundos_remunerados'] / model['ativos_rentaveis']

In [17]:
# Non-Interest Bearing Deposits (nibd) = Depósitos Não Remunerados / Total de Ativos Operacionais

model['ativo_operacional'] = model['ativo_realizavel'] + model['ativo_permanente']
model['nibd'] = model['depositos_a_vista'] / model['ativo_operacional']




In [18]:
# Custos Operacionais (opc)
'''
Despesas de Captação (8.1.1.00.00-8)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
Despesas de Arrendamento Mercantil (8.1.3.00.00-4)
Despesas de Câmbio (8.1.4.00.00-7)
Despesas com Títulos e Valores Mobiliários e Instrumentos Financeiros Derivativos (8.1.5.00.00-0)
Despesas Administrativas (8.1.7.00.00-6)
Despesas de Provisões Operacionais (8.1.8.30.00-0)
Outras Despesas Operacionais (8.1.9.99.00-6)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001', '81300004', '81400007', '81500000', '81700006', '81830000', '81999006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

custos_operacionais = pd.read_sql_query(query, conn)
conn.close()

custos_operacionais['cnpj'] = custos_operacionais['cnpj'].astype(int)
custos_operacionais = custos_operacionais.rename(columns={'saldo_total_agregado': 'custos_operacionais'})

model = pd.merge(
    model,
    custos_operacionais,
    on=['data', 'cnpj'],
    how='left'
)

model['opc'] = model['custos_operacionais'] / model['ativo_total']


In [19]:
# Ativos de Alta Liquidez (Caixa e Equivalentes de Caixa)
'''
Caixa (1.1.1.00.00-9)
Depósitos Bancários (1.1.2.00.00-2)
Reservas Livres (1.1.3.00.00-5)
Aplicações em Operações Compromissadas (1.2.1.00.00-8)
Aplicações em Depósitos Interfinanceiros (1.2.2.00.00-1)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11200002', '11300005', '12100008', '12200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

ativos_liquidos = pd.read_sql_query(query, conn)
conn.close()

ativos_liquidos['cnpj'] = ativos_liquidos['cnpj'].astype(int)
ativos_liquidos = ativos_liquidos.rename(columns={'saldo_total_agregado': 'ativo_liquido'})

model = pd.merge(
    model,
    ativos_liquidos,
    on=['data', 'cnpj'],
    how='left'
)


In [20]:
# Passivos de Curto Prazo (Obrigações Exigíveis no Circulante)
'''
Depósitos à Vista (4.1.1.00.00-0)
Obrigações por Operações Compromissadas (4.2.0.00.00-6)
Depósitos Interfinanceiros (4.1.3.00.00-6)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('41100000', '42000006', '41300006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

passivo_curto_prazo = pd.read_sql_query(query, conn)
conn.close()

passivo_curto_prazo['cnpj'] = passivo_curto_prazo['cnpj'].astype(int)
passivo_curto_prazo = passivo_curto_prazo.rename(columns={'saldo_total_agregado': 'passivo_curto_prazo'})

model = pd.merge(
    model,
    passivo_curto_prazo,
    on=['data', 'cnpj'],
    how='left'
)

In [21]:
# Liquidez (liquid)

model['liquid'] = model['ativo_liquido'] / model['passivo_curto_prazo']


In [22]:
# Renda de Serviços - Conta 71700009

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    balancetes
WHERE
    "CONTA" = '71700009'
"""

renda_servicos = pd.read_sql_query(query, conn)

conn.close()

renda_servicos = renda_servicos[['data', 'cnpj', 'SALDO']]
renda_servicos['cnpj'] = renda_servicos['cnpj'].astype(int)
renda_servicos = renda_servicos.rename(columns={'SALDO': 'renda_servicos'})

model = pd.merge(
    model,
    renda_servicos,
    on=['data', 'cnpj'],
    how='left'
)

In [23]:
# razão entre receitas de serviços e o total de receitas operacionais (servr)

model['servr'] = model['renda_servicos'] / model['receita_operacional']

In [24]:
# Receitas de Juros e Similares (Rendas)
'''
Rendas De Operacoes De Credito - Conta 7.1.1.00.00-1
Rendas de Aplicações Interfinanceiras de Liquidez (7.1.4.00.00-0)
Rendas de Arrendamento Mercantil (7.1.2.00.00-4)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71100001', '71400000', '71200004')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

receita_juros = pd.read_sql_query(query, conn)
conn.close()

receita_juros['cnpj'] = receita_juros['cnpj'].astype(int)
receita_juros = receita_juros.rename(columns={'saldo_total_agregado': 'receita_juros'})

model = pd.merge(
    model,
    receita_juros,
    on=['data', 'cnpj'],
    how='left'
)


In [25]:
# Ativos Remunerados Médios (Média do Balanço)
'''
Operações de Crédito (1.6.0.00.00-1)
Títulos e Valores Mobiliários (1.3.0.00.00-4)
Arrendamento Mercantil (1.7.0.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16000001', '13000004', '17000001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

saldo_credito = pd.read_sql_query(query, conn)
conn.close()

saldo_credito['cnpj'] = (saldo_credito['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
saldo_credito.dropna(subset=['cnpj'], inplace=True)
saldo_credito['cnpj'] = saldo_credito['cnpj'].astype(int)
saldo_credito = saldo_credito.rename(columns={'saldo_total_agregado': 'ativos_remunerados'})

model = pd.merge(
    model,
    saldo_credito,
    on=['data', 'cnpj'],
    how='left'
)


In [26]:
# taxa_empréstimos = receita de crédito / operações de crédito

model['taxa_emprestimos'] = model['receita_juros'] / model['ativos_remunerados']

In [27]:
# Despesas de Juros e Encargos (Custos de Captação)
'''
Despesas de captação (81100008)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

despesa_captacao = pd.read_sql_query(query, conn)
conn.close()

despesa_captacao['cnpj'] = despesa_captacao['cnpj'].astype(int)
despesa_captacao = despesa_captacao.rename(columns={'saldo_total_agregado': 'despesa_captacao'})
despesa_captacao['despesa_captacao'] = despesa_captacao['despesa_captacao'] * (-1)

model = pd.merge(
    model,
    despesa_captacao,
    on=['data', 'cnpj'],
    how='left'
)


In [28]:
# Outras despesas (Metodologia BCB)
'''
Despesas Administrativas (8.1.7.00.00-6)
Aprovisionamentos E Ajustes Patrimoniais (8.1.8.00.00-9)
Imposto de Renda (8.9.4.00.00-9)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81700006', '81800009', '86400009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

outras_despesas = pd.read_sql_query(query, conn)
conn.close()

outras_despesas['cnpj'] = (outras_despesas['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
outras_despesas.dropna(subset=['cnpj'], inplace=True)
outras_despesas['cnpj'] = outras_despesas['cnpj'].astype(int)
outras_despesas = outras_despesas.rename(columns={'saldo_total_agregado': 'outras_despesas'})

model = pd.merge(
    model,
    outras_despesas,
    on=['data', 'cnpj'],
    how='left'
)


In [29]:
# Saldo de captações
'''
DEPÓSITOS DE POUPANÇA (4.1.2.00.00-3)
DEPÓSITOS INTERFINANCEIROS (4.1.3.00.00-6)
DEPÓSITOS A PRAZO (4.1.5.00.00-2)
'''

conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('41200003', '41300006', '41500002')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

saldo_captacao = pd.read_sql_query(query, conn)
conn.close()

saldo_captacao['cnpj'] = saldo_captacao['cnpj'].astype(int)
saldo_captacao = saldo_captacao.rename(columns={'saldo_total_agregado': 'saldo_captacao'})

model = pd.merge(
    model,
    saldo_captacao,
    on=['data', 'cnpj'],
    how='left'
)


In [30]:
# taxa_captacao = despesa de captação / saldo de captacao

model['taxa_captacao'] = model['despesa_captacao'] / model['saldo_captacao']

In [35]:
# spread = taxa_emprestimos - taxa_captacao
model['taxa_spread'] = model['taxa_emprestimos'] - model['taxa_captacao']

model['teste_spread'] = (model['receita_juros'] - model['despesa_captacao']) / model['ativos_remunerados']


In [32]:
# Padronização da data e remoção de NaNs

model = model.dropna(how='any')
model['data'] = pd.to_datetime(model['data'], format='%Y-%m').dt.strftime('%Y-%m')


In [36]:
import numpy as np

matriz_correlacao = np.corrcoef(model['teste_spread'], model['taxa_spread'])
print(matriz_correlacao)

[[1.         0.58629737]
 [0.58629737 1.        ]]


In [37]:
# Modelo - Primeiro Estágio
import statsmodels.api as sm

X = model[['nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr']]
y = model['teste_spread']

stage1 = sm.OLS(y, sm.add_constant(X)).fit()
results_stage1 = stage1
print(results_stage1.summary())


                            OLS Regression Results                            
Dep. Variable:           teste_spread   R-squared:                       0.231
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     60.30
Date:                Tue, 21 Oct 2025   Prob (F-statistic):           5.30e-55
Time:                        17:43:18   Log-Likelihood:                 2947.5
No. Observations:                1011   AIC:                            -5883.
Df Residuals:                    1005   BIC:                            -5853.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0542      0.004     13.263      0.0

In [69]:
# Dados do IPEA

import ipeadatapy as ipea
 
desemprego = ipea.timeseries('PNADC12_TDESOC12')
desemprego = desemprego.reset_index(drop=False)
desemprego = desemprego[['DATE', 'VALUE ((%))']]
desemprego = desemprego.rename(columns={'DATE': 'data', 'VALUE ((%))': 'desemprego'})
desemprego['data'] = desemprego['data'].dt.strftime('%Y-%m')

model = model.merge(desemprego, on='data', how='left')